# Skyler's Salary Prediction Portfolio

## DEFINE

### Make salary predictions on each role with help from mean-squared-error (MSE) models

In [3]:
''' this script defines a problem and discovers the data'''

__author__ = 'Skyler Bullard, Business Analyst'
__contactinfo__ = 'skyler.bullard.r@gmail.com'